## 1. Propiedades  de [NGC 121](http://ned.ipac.caltech.edu/cgi-bin/objsearch?search_type=Obj_id&objid=3284737&objname=1&img_stamp=YES&hconst=73.0&omegam=0.27&omegav=0.73&corr_z=1)
1. Tiene las siguientes coordenadas ecuatoriales (J2000):  ($\alpha=$00h26m48.25s, $\delta=$-71d32m08.4s)
2. Su distancia es de apróximadamente 0.061 Mpc
3. Tiene un [tamaño angular](http://simbad.u-strasbg.fr/simbad/sim-id?Ident=NGC+0121&QueryType=ned) de 3'.8
4. Tiene un [índice de metalicidad](https://arxiv.org/pdf/1002.4301v2.pdf) de [Fe/H]=-1.5.
5. Para los [filtros B y V](http://simbad.u-strasbg.fr/simbad/sim-id?Ident=NGC+121) tiene las siguientes magnitudes: B=11.2, V=11.24.

## 2. Hubble Legacy Archive e images.
**ACS** es la sigla **A**dvanced **C**amera for **S**urvey, la cual detecta longitudes de onda desde el ultravioleta lejano hasta el infrarrojo cercano. Este instrumento contiene un trío de cámaras, entre ellas la **HRC** (**H**igh **R**esolution **C**hannel), la cual fue diseñada para obtener imagenes detalladas regiones interiores de de las galaxias, como también de cúmulos (https://www.spacetelescope.org/about/general/instruments/acs/).

Los filtros utilizados:
1. **F555W:** el más cercano en el sistema Johnson es el filtro V, que corresponde al verde.
2. **F814W:** el más cercano en el sistema Johnson es el filtro I, que corresponde al infrarrojo cercano.

Cada imagen posee cinco extensiones:
1. [**PRIMARY:**](http://www.stsci.edu/hst/acs/documents/handbooks/currentDHB/acs_Ch23.html) es un header global.
2. [**SCI:**](http://www.stsci.edu/hst/acs/documents/handbooks/currentDHB/acs_Ch23.html) *science image.* Contiene la información que queremos analizar.   
3. [**WHT:**]((http://www.stsci.edu/hst/HST_overview/drizzlepac/documents/handbooks/drizzlepac.pdf)) *weight image.* Contiene el peso cada pixel, que corresponde al inverso de la varianza.
4. [**CTX:**](http://www.stsci.edu/hst/HST_overview/drizzlepac/documents/handbooks/drizzlepac.pdf) *context image.* Contiene un mapa de las imagenes y un registro de cuál imagen contribuye a cada pixel.
5. [**EXP:**](http://www.stsci.edu/hst/HST_overview/documents/multidrizzle/ch55.html)  Contiene el tiempo de exposición de cada pixel.

En esta ocasión nos interesan las extensiones **SCI** y **WHT**:

In [ ]:
from astropy.io import fits

names = ['f555w_NRI.fits', 'f814w_NRI.fits']  # Nombres de los archivos, por si se necesitan cambiar.
for name in names:
    hdulist = fits.open(name)
    fits.writeto("sci_{}".format(name), hdulist[1].data, hdulist[1].header, clobber=True)
    fits.writeto("inver_{}".format(name), hdulist[2].data, hdulist[2].header, clobber=True)
    hdulist.close()

Ahora que tenemos las imagenes generadas podemos SExtractear. Para ello vamos a configurar nuestro input (configuration.sex):
1. [**GAIN:**](http://www.stsci.edu/hst/acs/documents/handbooks/currentDHB/acs_Ch48.html) 2
2. **DETECT_TYPE:** CCD
3. MAG_ZEROPOINT
4. [**PIXEL_SCALE:**](http://www.stsci.edu/hst/acs/documents/handbooks/cycle20/c02_cycle203.html) 0.025
5.

Ahora que tenemos los catálogos, debemos hacer un match:

In [ ]:
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.io import ascii
import numpy as np

names = ['cat_f555.cat', 'cat_f814.cat']  # Nombres de los catálogos
skies = []  # Lista de SkyCoord.
cats = [] # Lista de catálogos.

for name in names:
    data = ascii.read(name)
    cats.append(data)
    skies.append(SkyCoord(ra=data['ALPHA_J2000'], dec=data['DELTA_J2000'], unit='deg'))

# Obtenemos los matches. Se busca el más cercano de cada estrella en skies[0] en skies[1].
idx, d2d, d3d = SkyCoord.match_to_catalog_sky(skies[0], skies[1])
# Creamos una lista de tuplas que relaciona ambos catálogos.
matches = list(zip(range(0, len(idx)), idx, d2d))
# Quitamos los que están repetidos:
temp_matches = []
seconds = set()
matches.sort(key=lambda x: x[2], reverse=True)
for x in matches:
    if x[1] not in seconds:
        seconds.add(x[1])
        temp_matches.append(x)
matches = temp_matches

#matches = list(filter(lambda x: x[2]==0, matches))

# Ahora extraemos de los catálogos.
idx1, idx2, d2d = zip(*matches)
cats[0] = cats[0][list(idx1)]
cats[1] = cats[1][list(idx2)]

f555 = list(map(lambda x: -2.5*np.log10(x), cats[0]['FLUX_APER']))
f814 = list(map(lambda x: -2.5*np.log10(x), cats[1]['FLUX_APER']))


    

Ahora que hicimos el match, vamos a calibrar las magnitudes en el sistema VEGAMAG (el cual está definido para que tenga magnitud 0 para la estrella Vega, de este modo el punto zero corresponde a la magnitud de Vega):

$$\text{m}_{v} = -2.5\log_{10}(\text{f})+\text{m}_0-\text{apcor}-\text{A}_v$$
donde:
* $\text{m}_0$ es el *zeropoint*. Éste nos define el 0 de nuestras magnitudes.
* $\text{apcor}$ es la *aperture correction*. Debido a que la apertura no toma toda la luz de la estrella, es necesario agregar esta corrección. En este caso se utilizó la de 0''.15.
* $\text{A}_v$ es el *reddening*. Éste corresponde a la desviación, debida por el polvo interestelar, que sufre la luz, desviando en mayor medida los colores azules.

De [**Sirianni et al (2005)**](http://iopscience.iop.org/article/10.1086/444553/pdf) tenemos los siguientes valores para los $\text{apcor}$ y los $\text{m}_0$. Los valores de $\text{A}_v$ son de [**NED**](http://ned.ipac.caltech.edu/cgi-bin/objsearch?search_type=Obj_id&objid=3284737&objname=1&img_stamp=YES&hconst=73.0&omegam=0.27&omegav=0.73&corr_z=1):

|   **Filtro**  |$\text{m}_0$|$\text{apcor}$|$\text{A}_v$|
|:-------------:|:----------:|:------------:|:----------:|
|   **F555W**   |    25.255  |     0.241    |    0.103   |
|   **F814W **  |    24.849  |     0.425    |	 0.056   |

Aplicando esto obtenemos el CMD:

In [ ]:
#matplotlib inline
import matplotlib.pyplot as plt

# Corrección para F555w y F814w
mag = [f555, f814]
m_0 = [25.255, 24.849]
apcor = [0.241, 0.425]
a_v = [0.103, 0.056]

for idx in [0, 1]:
    mag[idx] = list(map(lambda x: x+m_0[idx]-apcor[idx]-a_v[idx], mag[idx]))

axis_x = list(map(lambda x: x[0]-x[1], zip(mag[0], mag[1])))
axis_y = mag[0]

plt.scatter(axis_x, axis_y, marker='o', c='Black', s=1)
plt.xlabel('F555 - F814')
plt.ylabel('F555')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()